In [8]:
import ibis
from ibis import _
import pathlib
import os
from osgeo import gdal
from cng.utils import *
from cng.h3 import *

gdal.DontUseExceptions()
os.environ['AWS_VIRTUAL_HOSTING'] = 'FALSE'

# from cng.zonal import * 
con = ibis.duckdb.connect("duck.db",extensions = ["spatial", "h3"])
install_h3()
set_secrets(con)

input_url = "/vsicurl/https://minio.carlboettiger.info/public-wetlands/GLWD_v2_0/GLWD_v2_0_combined_classes/GLWD_v2_0_main_class.tif"
#con.raw_sql("SET threads=1;") # Reducing threads reduces RAM use of duckdb
#con.raw_sql("SET memory_limit = '58GB';") # Reducing threads reduces RAM use of duckdb

In [25]:
grid = (con
.read_parquet("s3://public-grids/hex/h0.parquet")
.mutate(h0 = _.h0.lower())
.order_by(_.h0)
.mutate(i = ibis.row_number())
)

df = grid.execute()


In [26]:
( grid
  .filter(_.h0=="8029fffffffffff")
  .execute()
)

,h0,geom,i
0,8029fffffffffff,"POLYGON ((-121.336628 28.653019, -110.257485 3...",20


In [ ]:
%%time

#set_secrets(con, "", "", "s3.amazonaws.com", "overturemaps-us-west-2", 'vhost')
#overture = con.read_parquet('s3://overturemaps-us-west-2/release/2024-11-13.0/theme=divisions/type=division_area/*', filename=True, hive_partitioning=1)
#wkt = overture.filter(_.subtype=="region").filter(_.region == "US-CA").select(_.geometry).execute().geometry[0].wkt

# CA in 17 seconds
#dest = "test_wetlands.xyz"
#gdal.Warp(dest, input_url, dstSRS = 'EPSG:4326', cutlineWKT = wkt, cropToCutline = True)

In [27]:
%%time
i = 20
wkt = df.geom[i]
h0 = df.h0[i]
print(h0)
zoom = 8
gdal.Warp("tmp-wetlands.xyz", input_url, dstSRS = 'EPSG:4326', cutlineWKT = wkt, cropToCutline = True)


8029fffffffffff


Warning 1: the source raster dataset has a SRS, but the cutline features
not.  We assume that the cutline coordinates are expressed in the destination SRS.
If not, cutline results may be incorrect.


CPU times: user 1min 31s, sys: 5.49 s, total: 1min 36s
Wall time: 1min 37s


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x73ed60eef9f0> >

In [28]:
%%time
(con
  .read_csv("tmp-wetlands.xyz", 
            delim = ' ', 
            columns = {'X': 'FLOAT', 'Y': 'FLOAT', 'Z': 'INTEGER'})
  .mutate(h0 = h3_latlng_to_cell_string(_.Y, _.X, zoom),
          h8 = h3_latlng_to_cell_string(_.Y, _.X, zoom))
 .to_parquet("test.parquet")
  #.to_parquet(f"s3://public-carbon/hex/vulnerable-carbon/h0={h0}/vulnerable-total-carbon-2018-h{zoom}.parquet")
)
pathlib.Path("tmp-wetlands.xyz").unlink()

con.read_parquet("test.parquet")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 48.2 s, sys: 2.48 s, total: 50.7 s
Wall time: 4.93 s


DatabaseTable: ibis_read_parquet_ny77fge55rhu7ev64mhq34yzzq
  X  float32
  Y  float32
  Z  int32
  h0 string
  h8 string

In [33]:
con.read_parquet("test.parquet").group_by(_.Z).agg(n = _.Z.count()).order_by(_.n).execute()

,Z,n
0,5,3127
1,20,3637
2,16,4609
3,29,5990
4,10,6190
5,8,6310
6,17,13229
7,18,13750
8,33,14765
9,31,21761


In [ ]:

#see `gdal.WarpOptions?` for details. Also see resampling options in warper for large data
# NOTE! dest given before input!
for i in range(df.shape[0]):
    wkt = df.geom[i]
    h0 = df.h0[i]
    zoom = 8
    print(h0)

    try:
        gdal.Warp("tmp-carbon.xyz", input_url, dstSRS = 'EPSG:4326', cutlineWKT = wkt, cropToCutline = True)
    
        (con
          .read_csv("tmp-carbon.xyz", 
                    delim = ' ', 
                    columns = {'X': 'FLOAT', 'Y': 'FLOAT', 'Z': 'INTEGER'})
          .mutate(h0 = h3_latlng_to_cell_string(_.Y, _.X, zoom),
                  h8 = h3_latlng_to_cell_string(_.Y, _.X, zoom))
          .mutate(Z = ibis.ifelse(_.Z == 65535, None, _.Z)) 
          .to_parquet(f"s3://public-wetlands/hex/vulnerable-carbon/h0={h0}/vulnerable-total-carbon-2018-h{zoom}.parquet")
        )
        pathlib.Path("tmp-carbon.xyz").unlink()
    except Exception as e:
        print(f"Error processing item {i}: {e}")
    



In [ ]:
con.read_parquet("s3://public-carbon/hex/vulnerable-carbon/")